In [1]:
# Standard Python modules
import os, sys
import numpy as np
import pandas as pd
import xarray as xr
import datetime as dt
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

ERROR 1: PROJ: proj_create_from_database: Open of /home/dnash/miniconda3/envs/SEAK-impacts/share/proj failed


In [2]:
# Set up paths

path_to_data = '/expanse/nfs/cw3e/cwp140/'     # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [3]:
fname = os.path.join(path_to_data, 'preprocessed/SEAK-WRF-PCPT/PCPT_quantiles.nc')
ds = xr.open_dataset(fname)
# ds = ds.sel(quantile=0.95)
ds

<xarray.Dataset>
Dimensions:   (quantile: 3, y: 250, x: 320)
Coordinates:
    lat       (y, x) float32 ...
    lon       (y, x) float32 ...
  * quantile  (quantile) float64 0.0 0.75 0.95
Dimensions without coordinates: y, x
Data variables:
    PCPT      (quantile, y, x) float64 ...

In [4]:
## bounding boxes for composite analysis
bboxes = [[-140.5, -137., 58.25, 60.75], # Yakutat (Northern Coastal)
          [-136.5, -134.75, 56.0, 58.25], # Sitka / Hoonah (Central Coastal)
          [-137.0, -134.75, 58.25, 60.5], # Skagway/Klukwan/Haines (Northern Inner Channel)
          [-134.75, -132.5, 56.75, 58.5], # Juneau/Hoonah/Gustavas (Central Inner Channel)
          [-134.25, -132.5, 54.25, 56.75], # Craig (Southern Coastal)
          [-132.5, -131.25, 54.5, 56.75]] # Kasaan (Southern Inner Channel)

zone_lst = ['northern_coastal', 'central_coastal', 'northern_inner_channel', 'central_inner_channel', 'southern_coastal', 'southern_inner_channel']

In [5]:
## open precipitation and ivt dfs
## append precip to each community IVT df
option = 'a'
temporal_res = 'daily'
community_lst = ['Hoonah', 'Skagway', 'Klukwan', 'Yakutat', 'Craig', 'Kasaan']
varname = 'PCPT' # 'PCPT' or 'UV'

fname_pattern = path_to_data + 'preprocessed/SEAK-WRF-{0}/WRFDS_{0}_*.nc'.format(varname)
wrf = xr.open_mfdataset(fname_pattern, combine='by_coords', engine='netcdf4')
if temporal_res == 'hourly':
    wrf = wrf
elif (temporal_res == 'daily') & (varname == 'PCPT'):
    wrf = wrf.resample(time="1D").sum('time') # resample WRF data to be mm per day
    
elif (temporal_res == 'daily') & (varname == 'UV'):
    wrf = wrf.sel(lev='1000')
    wrf = wrf.resample(time="1D").mean('time') # resample WRF data to be m s-1

## subset to AR days only
## append AR data
fname = '../out/SEAK_ardates_{0}.csv'.format(temporal_res)
ar_df = pd.read_csv(fname) # read in AR dates
idx = (ar_df.AR == 1)
ar_df = ar_df.loc[idx]
# reset the index as "time"
ar_df = ar_df.set_index(pd.to_datetime(ar_df['Unnamed: 0']))
## subset dates to Jan 2000 - Dec 2019
idx = (ar_df.index.year >= 2000) & (ar_df.index.year <= 2019)
ar_df = ar_df.loc[idx]
ar_dates = ar_df.index.values

wrf = wrf.sel(time=ar_dates)
wrf

<xarray.Dataset>
Dimensions:  (time: 2463, y: 250, x: 320)
Coordinates:
  * time     (time) datetime64[ns] 2000-01-06 2000-01-08 ... 2019-08-26
    lat      (y, x) float32 dask.array<chunksize=(250, 320), meta=np.ndarray>
    lon      (y, x) float32 dask.array<chunksize=(250, 320), meta=np.ndarray>
Dimensions without coordinates: y, x
Data variables:
    PCPT     (time, y, x) float32 dask.array<chunksize=(1, 250, 320), meta=np.ndarray>
Attributes:
    contact:  rtladerjr@alaska.edu
    info:     Alaska CASC
    data:     Downscaled CFSR
    format:   version 1
    date:     Sat Dec  8 15:40:16 AKST 2018

In [6]:
%%time
## get the total number of grids in each subregion
total_grid_ct = []
for i, ext in enumerate(bboxes):
    idx = (wrf.lat >= ext[2]) & (wrf.lat <= ext[3]) & (wrf.lon >= ext[0]) & (wrf.lon <= ext[1])
    mask = wrf.where(idx)
    tmp = mask.isel(time=0)
    grid_ct = tmp.PCPT.count(['y', 'x']).compute()
    total_grid_ct.append(grid_ct.values)

CPU times: user 25.2 s, sys: 189 ms, total: 25.3 s
Wall time: 25.5 s


In [7]:
%%time
## subset WRF to grids that fall within the percentile categories
impact_level = 'high'

if impact_level == 'high':
    ## use where statement to get dates where precip >95th percentile
    wrf = wrf.where(wrf.PCPT >= ds.PCPT.sel(quantile=0.95))
    wrf = wrf.compute()
elif impact_level == 'medium':
    ## use where statement to get dates where precip >75th percentile and <95th percentile
    idx = (wrf.PCPT >= ds.PCPT.sel(quantile=0.75)) & (wrf.PCPT <= ds.PCPT.sel(quantile=0.95))
    wrf = wrf.where(idx)
    wrf = wrf.compute()
elif impact_level == 'low':
    ## use where statement to get dates where precip <75th percentile
    wrf = wrf.where(wrf.PCPT <= ds.PCPT.sel(quantile=0.75))
    wrf = wrf.compute()

CPU times: user 29.4 s, sys: 24.6 s, total: 54 s
Wall time: 44.6 s


In [8]:
%%time
## now iterate through all the zones and compute the number of grids that fall within the percentile zones
## save as a csv
for i, ext in enumerate(bboxes):
    ## temporarily subset to the region
    idx = (wrf.lat >= ext[2]) & (wrf.lat <= ext[3]) & (wrf.lon >= ext[0]) & (wrf.lon <= ext[1])
    mask = wrf.where(idx)
    
    ## get number of grids where prec >=95th percentile
    grid_ct = mask.PCPT.count(['y', 'x'])
    grid_perc = (grid_ct/total_grid_ct[i])*100
    final_dates = grid_perc.where(grid_perc >= 50., drop=True).time.values

    d = {'date': final_dates}
    df = pd.DataFrame(data=d)
    df.to_csv('../out/{0}_{1}_impact_precip_dates.csv'.format(zone_lst[i], impact_level))

CPU times: user 1.87 s, sys: 617 ms, total: 2.48 s
Wall time: 2.68 s
